In [6]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

# todo: find better way to describe strategy.. pct allocated to asset, etc.

#------------------------------
#-- 0. Read in Assets
#------------------------------
df = pd.read_csv('D1_assets.csv')

#------------------------------
#-- 0. Assign Strategies
#------------------------------

# buy 1 at start
PID = 'S03'
START_DATE = '2013-01-01' # to handle diff assets start at diff date
PRICE_COL = '"Adj Close"' # SQL needs double quotes on field name w space

# find the first 01 day for each stock
q = """SELECT 
      AID
     ,min(Date) as first_01_day
    FROM df
    WHERE strftime('%d', Date) = '01'
    GROUP BY AID
    ;"""
f01_df = pysqldf(q) # not used, to be used later

q = """SELECT 
      '{pid}' as PID
     ,df.Date
     ,df.AID
     ,df."Adj Close"
     ,CASE WHEN Date=first_01_day THEN 1000/"Adj Close" ELSE 0 END as qty_chg
    FROM df
    LEFT JOIN f01_df f01 on f01.AID=df.AID
    WHERE Date >= '{start_date}'
      AND df.AID in ('BRK-B', 'SPY')
    ORDER BY PID, df.AID, Date
    ;""".format(pid=PID, start_date=START_DATE)

df = pysqldf(q)

#------------------------------
#-- 0. Output to csv
#------------------------------
# Key: PID AID Date
df.to_csv('S03 BRK-B SPY.csv',index=False)

In [7]:
df.head(25)

,PID,Date,AID,Adj Close,qty_chg
0,S03,2014-03-03,BRK-B,116.059998,0.000000
1,S03,2014-03-04,BRK-B,118.570000,0.000000
2,S03,2014-03-05,BRK-B,118.980003,0.000000
3,S03,2014-03-06,BRK-B,121.199997,0.000000
4,S03,2014-03-07,BRK-B,122.669998,0.000000
5,S03,2014-03-10,BRK-B,124.099998,0.000000
6,S03,2014-03-11,BRK-B,124.690002,0.000000
7,S03,2014-03-12,BRK-B,125.169998,0.000000
8,S03,2014-03-13,BRK-B,123.610001,0.000000
9,S03,2014-03-14,BRK-B,122.510002,0.000000
